In [80]:
import tensorflow as tf
import pandas as pd
import numpy as np

import os
import time

import csv
csv.field_size_limit(1000000)

lyrics = []
with open('lyrics.csv') as csvfile:
    r = csv.reader(csvfile)
    for i, line in enumerate(r):
        if line[0] == "Stevie Wonder":
            lyrics += line[1]


print(len(lyrics))
vocab = sorted(set(lyrics))

201411


In [81]:
char_to_idx = {char:idx for idx, char in enumerate(vocab)}
idx_to_char = np.array(vocab)
int_text = np.array([char_to_idx[char] for char in lyrics])

seq_length = 100
len_examples = len(lyrics)
char_dataset = tf.data.Dataset.from_tensor_slices(int_text)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# for item in sequences.take(2):
#     print(repr(''.join(idx_to_char[item.numpy()])))

def split_text(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_text)

# for input_example, target_example in dataset.take(1):
#     print ('Input data: ', repr(''.join(idx_to_char[input_example.numpy()])))
#     print ('Target data:', repr(''.join(idx_to_char[target_example.numpy()])))

# for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
#     print("Step {:4d}".format(i))
#     print("  input: {} ({:s})".format(input_idx, repr(idx_to_char[input_idx])))
#     print("  expected output: {} ({:s})".format(target_idx, repr(idx_to_char[target_idx])))

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# MODEL

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model =  tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    
    return model

model = build_model(vocab_size=vocab_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (64, None, 256)           18432     
_________________________________________________________________
gru_16 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_16 (Dense)             (64, None, 72)            73800     
Total params: 4,030,536
Trainable params: 4,030,536
Non-trainable params: 0
_________________________________________________________________


In [82]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print('sampled indices: ', sampled_indices)

print("Input: \n", repr("".join(idx_to_char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx_to_char[sampled_indices])))


(64, 100, 72) # (batch_size, sequence_length, vocab_size)
sampled indices:  [ 8 59 21 10 38 12 67 23 48 18  2 21  6 62 22 37 27 54 62 46  2  0 17  9
 59 20 26 61 64 53 43 70 18 22 25 40 47 23 50 32  8 39 33 66 52 45 55 20
 30 12 21  1 29 60 67 27 23 52 70 59 70 41 33  8 45 14 66 69 49 12 34 13
 54 42 48 21 54 56 61 37  7 51 21 18 28 11 36 32  0 29  5 29 18 49 36 18
  9 38 67 23]
Input: 
 "at lover's park  \nOne rainy April Monday  \nSmiling soft and warm as on  \nA clear December Sunday  \nW"

Next Char Predictions: 
 '-nC0T2vEc?!C)qDSIiqa!\n:.nBHpshZy?DGVbEeN-UOug]jBL2C KovIEgynyWO-]4uxd2P3iYcCikpS,fC?J1RN\nK(K?dR?.TvE'


In [83]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss: ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

Prediction shape:  (64, 100, 72)  # (batch_size, sequence_length, vocab_size)
scalar_loss:  4.2786756


In [84]:
# CHECKPOINTS

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [85]:
# TRAIN

EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 31 steps
Epoch 1/10
31/31 [==============================] - 51s 2s/step - loss: 3.8746
Epoch 2/10
31/31 [==============================] - 49s 2s/step - loss: 2.6578
Epoch 3/10
31/31 [==============================] - 49s 2s/step - loss: 2.2801
Epoch 4/10
31/31 [==============================] - 49s 2s/step - loss: 2.1307
Epoch 5/10
31/31 [==============================] - 50s 2s/step - loss: 2.0306
Epoch 6/10
31/31 [==============================] - 49s 2s/step - loss: 1.9358
Epoch 7/10
31/31 [==============================] - 49s 2s/step - loss: 1.8440
Epoch 8/10
31/31 [==============================] - 50s 2s/step - loss: 1.7551
Epoch 9/10
31/31 [==============================] - 49s 2s/step - loss: 1.6685
Epoch 10/10
31/31 [==============================] - 49s 2s/step - loss: 1.5861


In [86]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [87]:
import h5py

model.save('steviewonder.h5')

In [11]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char_to_idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.3

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx_to_char[predicted_id])

    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"We will "))

We will the sure  
We wance  
Some me and the sing  
Mo las a sand  
I wan the gon the seel  
I was seat it a still  
  
When't I wan the sure the sere to here  
  
Love is a beat a way  
  
I'm a can the  
The pant in the sare  
I  
What it a finger  
I wan't the wing the sight  
And you dange  
When I can the  
Then you  
Wher a sure it sore  
I was a me the sore that hear  
I wan't  
So lave a dan  
The stay  
When you  
I wan't you  
  
When you and the wan  
We was  
  
Le last a dan the sore and you don't you  
So love a meer a bean  
  
So see the seel  
And the sone  
I wan't you  
I want  
  
I'm a dand  
I wan the mere the  
The sing the sance  
And the sight  
I was the  
I wand  
When't have me the sime  
  
I was the me what I hear  
  
So han you  
  
I lave a beat a songe  
And you have me the  
The want  
  
I wan's every the sound  
Love a sing the sill a lither  
  
I'm a cone  
And I can your live a latt a  
And I was a can the me night  
I sand of the sand  
  
I wa